In [1]:
import numpy as np
import uproot
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, StackingRegressor
from sklearn.multioutput import MultiOutputRegressor
import sklearn
from random import random
from sklearn.model_selection import cross_validate
import matplotlib
from waveform_methods import binning
import pandas as pd

In [2]:
file = uproot.open('/home/andy/12360.root')
file2 = uproot.open('/home/andy/12362.root')

In [3]:
Xmax1 = file['MCPrimaryInfo']['ghMaxDepth'].array()
valueDepth = file['MCPrimaryInfo']['longNumCharged'].array()
Depth1 = file['MCPrimaryInfo']['longDepth'].array()
chi2_1 = file['CurvatureOnlyParams']['chi2_time'].array()
red1 = file['MCPrimaryInfo']['ghRedChiSqr'].array()
mass1 = [1 for i in range(len(red1))]

good = []
count = 0
for i in range(len(Xmax1)):
    if (red1[i]<300 and chi2_1[i]<5) and Xmax1[i]<900:
        good.append(1)
        count+=1
    else:
        good.append(0)
        
Xmax2 = file2['MCPrimaryInfo']['ghMaxDepth'].array()
valueDepth = file2['MCPrimaryInfo']['longNumCharged'].array()
Depth2 = file2['MCPrimaryInfo']['longDepth'].array()
chi2_2 = file2['CurvatureOnlyParams']['chi2_time'].array()
red2 = file2['MCPrimaryInfo']['ghRedChiSqr'].array()
mass2 = [4 for i in range(len(red2))]

good2 = []
for i in range(len(Xmax2)):
    if (red2[i]<300 and chi2_2[i]<5) and Xmax2[i]<650:
        good2.append(1)
        count+=1
    else:
        good2.append(0)
print(count)

172382


In [4]:
S125_1 = np.log10(file['LaputopParams']['s125'].array())
S125_2 = np.log10(file2['LaputopParams']['s125'].array())

In [5]:
A1 = file['CurvatureOnlyParams']['A'].array()
A2 = file2['CurvatureOnlyParams']['A'].array()
D1 = file['CurvatureOnlyParams']['D'].array()
D2 = file2['CurvatureOnlyParams']['D'].array()
N1 = file['CurvatureOnlyParams']['N'].array()
N2 = file2['CurvatureOnlyParams']['N'].array()
beta1 = file['LaputopParams']['beta'].array()
beta2 = file2['LaputopParams']['beta'].array()
zenith1 = file['Laputop']['zenith'].array()
zenith2 = file2['Laputop']['zenith'].array()
rlogl1 = file['CurvatureOnlyParams']['rlogl'].array()
nmini1 = file['CurvatureOnlyParams']['nmini'].array()
ndf1 = file['CurvatureOnlyParams']['ndf'].array()
llh1 = file['CurvatureOnlyParams']['llh'].array()
rlogl2 = file2['CurvatureOnlyParams']['rlogl'].array()
nmini2 = file2['CurvatureOnlyParams']['nmini'].array()
ndf2 = file2['CurvatureOnlyParams']['ndf'].array()
llh2 = file2['CurvatureOnlyParams']['llh'].array()

Xmax1_new = Xmax1
Xmax2_new = Xmax2

In [6]:
A = np.append(A1,A2)
D = np.append(D1,D2)
N = np.append(N1,N2)
S125 = np.append(S125_1,S125_2)
beta = np.log10(np.append(beta1,beta2))
chi2 = np.log10(np.append(chi2_1,chi2_2))
zenith = np.cos(np.append(zenith1,zenith2))
Xmax = np.append(Xmax1_new,Xmax2_new)
goodness = np.append(good,good2)
red = np.append(red1,red2)
mass = np.append(mass1,mass2)

In [7]:
print(np.corrcoef(np.array([A,D,N,S125,beta,zenith,Xmax,chi2,red,mass,goodness])))

[[ 1.          0.30749226 -0.05110983 -0.04803924  0.10285016  0.21427943
   0.03255202  0.77079182 -0.04239033 -0.10293111 -0.23799226]
 [ 0.30749226  1.         -0.0435993  -0.52563202  0.06855834  0.02395401
   0.00522173  0.18298388 -0.09536741 -0.14998897  0.0453969 ]
 [-0.05110983 -0.0435993   1.          0.08137576 -0.0860212  -0.10458563
  -0.01237658 -0.04203863  0.04476834  0.0769891  -0.03354958]
 [-0.04803924 -0.52563202  0.08137576  1.          0.07575187  0.19535921
   0.07220079 -0.14748136  0.17610724  0.04298084 -0.14193754]
 [ 0.10285016  0.06855834 -0.0860212   0.07575187  1.          0.41891977
   0.04970585 -0.12331223 -0.04467872 -0.20083333  0.15994521]
 [ 0.21427943  0.02395401 -0.10458563  0.19535921  0.41891977  1.
  -0.04087814 -0.05310425 -0.09956015  0.01339489  0.30870411]
 [ 0.03255202  0.00522173 -0.01237658  0.07220079  0.04970585 -0.04087814
   1.         -0.00276444  0.04444636 -0.19341011 -0.08236207]
 [ 0.77079182  0.18298388 -0.04203863 -0.14748136

In [8]:
input_variable = np.array([np.append(np.append(np.append(i,j),k),l) for i,j,k,l in zip(S125,A,zenith,chi2)])
output = list(zip(Xmax,goodness))

In [9]:
from sklearn.model_selection import train_test_split
seed = 7
test_size = 0.2
X_train, X_test, y_train_1, y_test_1 = train_test_split(input_variable,output , test_size=test_size, random_state=seed)

In [10]:
from scipy.stats import norm
y_train = list(zip(*y_train_1))[1]
y_test = list(zip(*y_test_1))[1]
for i in [3]:
    for j in [5]:
        rng = np.random.RandomState(1)
        regr_1 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=j,criterion='friedman_mse'),
                                n_estimators=i, random_state=rng,loss='square')
        regr_1.fit(X_train,y_train)
    
        y_1= regr_1.predict(X_test)
        y_2 = regr_1.predict(X_train)

        #mse = [(i-j)**2.0 for i,j in zip(y_1,y_test)]
        #mse2 = [(i-j)**2.0 for i,j in zip(y_2,y_train)]
        count_0 = 0
        count_02 = 0
        for value in y_train:
            if value==0:
                count_0+=1
        for value in y_test:
            if value==0:
                count_02+=1
        
        count = 0
        count2 = 0
        for value1,value2 in zip(y_1,y_test):
            if value1<0.5 and value2==0:
                count+=1

        for value1,value2 in zip(y_2,y_train):
            if value1<0.5 and value2==0:
                count2+=1

        print(count/count_02,count2/count_0,i,j)
        
        #residual = [i-j for i,j in zip(y_1,y_test)]
        #mean,std=norm.fit(residual)
        #print(mean,std,i,j)

0.7318361955085865 0.7272928502914484 3 5


In [11]:
check = Xmax>300
check2 = Xmax<900
new_check= []
for i,j in zip(check,check2):
    if i and j:
        new_check.append(True)
    else:
        new_check.append(False)
    
mask = regr_1.predict(input_variable)
mask_convert_pre = []
for i in mask:
    if i>=0.5:
        mask_convert_pre.append(True)
    else:
        mask_convert_pre.append(False)

mask_convert = []
for i,j in zip(new_check,mask_convert_pre):
    if i and j:
        mask_convert.append(True)
    else:
        mask_convert.append(False)

In [12]:
mask_new = []
import random
random.seed(1)
for i,j in zip(mask_convert,mass):
    value = random.random()
    if (j==1 and i) and value < 0.15:
        mask_new.append(False)
    else:
        mask_new.append(i)

In [13]:
print(np.corrcoef(np.array([A[mask_new],
                            D[mask_new],
                            N[mask_new],
                            S125[mask_new],
                            beta[mask_new],
                            zenith[mask_new],
                            Xmax[mask_new],
                            chi2[mask_new],
                            red[mask_new],
                            mass[mask_new],
                            goodness[mask_new]])))

[[ 1.          0.10495297 -0.53447646  0.23156825  0.1526491   0.36316278
   0.20176385  0.51469105 -0.01392814 -0.16843856  0.07060271]
 [ 0.10495297  1.         -0.05426723 -0.54634202 -0.00185964 -0.22000397
   0.02592712  0.01531053 -0.06612157 -0.18691821  0.06523613]
 [-0.53447646 -0.05426723  1.          0.13589256 -0.02080887 -0.07082137
  -0.09757153 -0.49230504  0.04904384  0.14651449 -0.07364571]
 [ 0.23156825 -0.54634202  0.13589256  1.          0.14479957  0.49004041
   0.2338944  -0.01138146  0.12933913  0.06367302 -0.12211743]
 [ 0.1526491  -0.00185964 -0.02080887  0.14479957  1.          0.28458703
   0.19742547 -0.14001288 -0.00445515 -0.17696301  0.04787336]
 [ 0.36316278 -0.22000397 -0.07082137  0.49004041  0.28458703  1.
  -0.07002358 -0.04168865 -0.01285383  0.0268446   0.17892384]
 [ 0.20176385  0.02592712 -0.09757153  0.2338944   0.19742547 -0.07002358
   1.          0.0086344   0.05969008 -0.66657325 -0.12386341]
 [ 0.51469105  0.01531053 -0.49230504 -0.01138146

In [14]:
input_variable2_old = np.array([np.append(i,j) for i,j in zip(D[mask_new],beta[mask_new])])
output_new_old = mass[mask_new]

import random
random.seed(1)
validation_x = []
validation_y = []
input_variable2 = []
output_new = []

for i in range(len(input_variable2_old)):
    if random.random() < 0.1:
        validation_x.append(input_variable2_old[i])
        validation_y.append(output_new_old[i])
    else:
        input_variable2.append(input_variable2_old[i])
        output_new.append(output_new_old[i])

In [15]:
from sklearn.model_selection import train_test_split
seed = 7
test_size = 0.2
X_train, X_test, y_train_1, y_test_1 = train_test_split(input_variable2,output_new , test_size=test_size, random_state=seed)

In [16]:
def convert_mass(X):
    output = []
    count1 = 0
    count2 = 0
    for i in X:
        if i < 2.5:
            output.append(1)
            count1+=1
        else:
            output.append(4)
            count2+=1
    return output

In [17]:
from scipy.stats import norm
'''
for i in range(1,100):
    for j in range(10,21):
        rng = np.random.RandomState(1)
        regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=j,criterion='friedman_mse'),
                                        n_estimators=i, random_state=rng,loss='square')
        regr_2.fit(X_train,y_train_1)
    
        new_1= regr_2.predict(X_test)
        new_2 = regr_2.predict(X_train)
        square = [(i-j)**2.0 for i,j in zip(new_1,y_test_1)]
        square2 = [(i-j)**2.0 for i,j in zip(new_2,y_train_1)]
        #print(np.mean(square),np.mean(square2),i,j)
        mass_predict = []
        for k in new_1:
            if k <=2.5:
                mass_predict.append(1)
            elif k>2.5:
                mass_predict.append(4)
                
        mass_predict2 = []
        for k in new_2:
            if k <=2.5:
                mass_predict2.append(1)
            elif k>2.5:
                mass_predict2.append(4)
                
        from sklearn.metrics import confusion_matrix
        cm=confusion_matrix(y_test_1,mass_predict)
        cm2 = confusion_matrix(y_train_1,mass_predict2)
        print(cm[0][0]/np.sum(cm[0]),cm[1][1]/np.sum(cm[1]),cm2[0][0]/np.sum(cm2[0]),cm2[1][1]/np.sum(cm2[1]),i,j)
'''

"\nfor i in range(1,100):\n    for j in range(10,21):\n        rng = np.random.RandomState(1)\n        regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=j,criterion='friedman_mse'),\n                                        n_estimators=i, random_state=rng,loss='square')\n        regr_2.fit(X_train,y_train_1)\n    \n        new_1= regr_2.predict(X_test)\n        new_2 = regr_2.predict(X_train)\n        square = [(i-j)**2.0 for i,j in zip(new_1,y_test_1)]\n        square2 = [(i-j)**2.0 for i,j in zip(new_2,y_train_1)]\n        #print(np.mean(square),np.mean(square2),i,j)\n        mass_predict = []\n        for k in new_1:\n            if k <=2.5:\n                mass_predict.append(1)\n            elif k>2.5:\n                mass_predict.append(4)\n                \n        mass_predict2 = []\n        for k in new_2:\n            if k <=2.5:\n                mass_predict2.append(1)\n            elif k>2.5:\n                mass_predict2.append(4)\n                \n        fro

In [18]:
rng = np.random.RandomState(1)
regr_3 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=10,criterion='friedman_mse'),
                            n_estimators=63, random_state=rng,loss='square')
regr_3.fit(X_train,y_train_1)
    
new_1= regr_3.predict(X_test)
new_2 = regr_3.predict(X_train)
mse = [(m-n)**2.0 for m,n in zip(new_1,y_test_1)]
mse2 = [(m-n)**2.0 for m,n in zip(new_2,y_train_1) ]
print(np.mean(mse),np.mean(mse2))

2.2481736182263674 2.2468737163769004


In [19]:
mass_predict = []
for i in new_1:
    if i <2.5:
        mass_predict.append(1)
    else:
        mass_predict.append(4)

In [20]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test_1,mass_predict))
cm=confusion_matrix(y_test_1,mass_predict)

[[12016   641]
 [10984  1197]]


In [21]:
print(cm[0][0]/np.sum(cm[0]),cm[1][1]/np.sum(cm[1]))

0.9493560875404914 0.09826779410557425


In [22]:
rng = np.random.RandomState(1)
regr_4 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=19,criterion='friedman_mse'),
                            n_estimators=65, random_state=rng,loss='square')
regr_4.fit(X_train,y_train_1)
    
new_3= regr_4.predict(X_test)
new_4 = regr_4.predict(X_train)

mse = [(m-n)**2.0 for m,n in zip(new_3,y_test_1)]
mse2 = [(m-n)**2.0 for m,n in zip(new_4,y_train_1) ]
print(np.mean(mse),np.mean(mse2),i,j)

2.2449837774607855 2.234356118626378 2.5003114035998255 4


In [23]:
mass_predict1 = []
for i in new_3:
    if i <2.5:
        mass_predict1.append(1)
    else:
        mass_predict1.append(4)

In [24]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test_1,mass_predict1))
cm=confusion_matrix(y_test_1,mass_predict1)

[[ 3121  9536]
 [ 1592 10589]]


In [25]:
print(cm[0][0]/np.sum(cm[0]),cm[1][1]/np.sum(cm[1]))

0.2465829185430987 0.8693046547902471


In [26]:
predict1 = regr_3.predict(validation_x)
    
mass_predict = []
for i in predict1:
    if i <2.5:
        mass_predict.append(1)
    else:
        mass_predict.append(4)
cm=confusion_matrix(validation_y,mass_predict)
    
predict2 = regr_4.predict(validation_x)
    
mass_predict = []
for i in predict2:
    if i <2.5:
        mass_predict.append(1)
    else:
        mass_predict.append(4)
cm1=confusion_matrix(validation_y,mass_predict)
print(cm[0][0]/np.sum(cm[0]),cm1[1][1]/np.sum(cm1[1]))

0.9155700477465937 0.8278532284965452


In [27]:
#import pickle
#pickle.dump(regr_3, open('proton_model.sav', 'wb'))
#pickle.dump(regr_4, open('iron_model.sav','wb'))